Write python code to do inverse kinematics of the leader or follower robotic arm given in https://github.com/AlexanderKoch-Koch/low_cost_robot/tree/main/simulation

In [1]:
import pybullet as p
import pybullet_data
import time
import numpy as np

# Connect to the physics server
p.connect(p.GUI)

# Set additional search path for URDF data
p.setAdditionalSearchPath(r"C:\Users\Administrator\arm\urdf")

# Load the URDF file
robot = p.loadURDF(r"arm.urdf", useFixedBase=True)

initial_base_pose = [0, 0, 0]
initial_base_orientation = p.getQuaternionFromEuler([-np.pi/2, 0, -np.pi/2])  # Rotate by 90 degrees around the z-axis
p.resetBasePositionAndOrientation(robot, initial_base_pose, initial_base_orientation)

# Set the initial joint positions
initial_joint_positions = [0.0, 0.0, 0.0, 0.0, 0.0]  # Adjust based on your robot's initial configuration
for i in range(p.getNumJoints(robot)):
    p.resetJointState(robot, i, initial_joint_positions[i])

# Add a camera to the scene
p.addUserDebugText('Camera view of the arm', [0.02, 0, 0.25], textColorRGB=[0,0,0], textSize=1)
p.resetDebugVisualizerCamera(cameraDistance=0.45, cameraYaw=45, cameraPitch=-45, cameraTargetPosition=[0, 0, 0])

# Simulate the robot
p.setGravity(0, 0, -9.81)
p.setRealTimeSimulation(1)

color2 = [0.851, 0.424, 0.024, 0.961]  # light
color1 = [0.161, 0.157, 0.137, 0.998]  # dark
current_color = color1

# Change colors of alternate joints
for i in range(0, p.getNumJoints(robot), 1):
    p.changeVisualShape(robot, i, rgbaColor=current_color)
    current_color = color2 if current_color == color1 else color1

def move_to_target(target_end_effector_position):
    # Calculate inverse kinematics to get joint positions
    joint_positions = p.calculateInverseKinematics(robot, 4, target_end_effector_position,target_ori, maxNumIterations=1000)
    # Move the arm to the calculated joint positions
    for i in range(p.getNumJoints(robot)):
        p.setJointMotorControl2(robot,i,p.POSITION_CONTROL,targetPosition=joint_positions[i],maxVelocity=0.7)

    # Print the joint positions after reaching the target
    print("Joint angles after moving to target:",joint_positions)

# Move to a target end-effector position (x, y, z)
target_end_effector_position = [-0.06484896781700908, -0.03385325078356215, 0.09274640062348348]
target_ori=[-0.47487614004185047, 0.23152471820976536, 0.8381955897065375, 0.1353407177214285]
move_to_target(target_end_effector_position)

time.sleep(7)
p.disconnect()  # Close the connection to PyBullet


Joint angles after moving to target: (2.943409995173882, -0.8238407385587269, -0.13475058174699264, 0.9048929849392646, -1.4068743963136143)
